# 대회 설명
- 화합물의 대사안정성 학습용 데이터 3,498종을 이용해 예측모델을 개발
- 개발한 모델로 경진용 데이터 483종 화합물을 이용하여 대사안정성 예측값을 제출

- 평가
 - 평가 산식 : 0.5 * RMSE(MLM) + 0.5 * RMSE(HLM)
 - Public score : 전체 테스트 데이터 중 약 35%
 - Private score : 전체 테스트 데이터

# 외부데이터 소스

- 데이터 받아올 수 있는 곳들 리스트 (화합물, 분자구조 데이터 등)

- PubChem: 화합물 및 생물학적 활성 데이터베이스로, 화합물의 구조, 물리적 및 화학적 특성, 바이오활성 등의 정보를 제공합니다.: https://pubchem.ncbi.nlm.nih.gov/

- ChEMBL: 생물학적 활성 정보를 가진 화합물 데이터베이스로, 화합물의 활성 여부와 관련 생물학적 실험 결과를 포함합니다. https://www.ebi.ac.uk/chembl/

- ZINC: 화합물 데이터베이스로, 다양한 화합물의 구조와 관련 데이터를 제공합니다. http://zinc.docking.org/

- ChemSpider: 화학 정보 데이터베이스로, 화합물의 구조, 물리화학적 특성 등을 검색 및 확인할 수 있습니다. http://www.chemspider.com/

- DrugBank: 의약품 및 화합물 정보 데이터베이스로, 화합물의 구조, 생물학적 활성, 약동학 등을 포함합니다 https://go.drugbank.com/

- ChEBI: 화합물 엔터티 데이터베이스로, 화합물의 화학적 및 생물학적 특성 정보를 포함합니다.https://www.ebi.ac.uk/chebi/

- PDB (Protein Data Bank): 단백질 및 분자의 3D 구조 정보 데이터베이스로, 화합물과의 상호 작용 정보도 포함할 수 있습니다.https://www.rcsb.org/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

import random
import os

DATA_PATH = '/content/drive/MyDrive/데이콘 캐글 컴페티션/2023신약개발/data/'
SEED = 42

In [ ]:
from sklearn.feature_selection import VarianceThreshold


In [ ]:
train = pd.read_csv(f"{DATA_PATH}train.csv")
test = pd.read_csv(f"{DATA_PATH}test.csv")
submission = pd.read_csv(f"{DATA_PATH}sample_submission.csv")

# 데이터 확인
- SMILES : 화합물 분자구조
- MLM/ HLM: 화합물의 대사안정성 지표 (인간, 쥐 - 대사되지 않고 남아있는 화합물의 양을 측정한 것) : 낮을 수록 안정성 좋은것
- AlogP : 화합물이 물-유기용매 사이에서 분배되는 정도 (로그파티션 계수)
- Molecular Weight: 분자량, 분자의 총 무게
- Num_H_Acceptors: 화합물의 수소 수용체 개수. 수소 원자가 수용체로 작동하는 원자를 의미
- Num_H_Donors : 화합물의 수소 공여체 개수. 수소 원자가 수소 결합을 형성할 수 있는 원자를 의미
- Num_RotatableBonds: 분자 내에서 회전이 가능한 결합 개수
- LogD : 화합물의 분배 계수. 로그 파티션 계수와 유사하나, 조금 다름. '어떻게 분배되나?'를 포함함
- Molecular_PolarSurfaceArea: 분자의 극성 표면 면적. 분자 내에서 극성 원자들이 차지하는 면적

In [ ]:
train.columns

Index(['id', 'SMILES', 'MLM', 'HLM', 'AlogP', 'Molecular_Weight',
       'Num_H_Acceptors', 'Num_H_Donors', 'Num_RotatableBonds', 'LogD',
       'Molecular_PolarSurfaceArea'],
      dtype='object')

In [ ]:
train.head()

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,117.37
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,73.47
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,62.45
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,92.60
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,42.43


In [ ]:
test.head()

,id,SMILES,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
0,TEST_000,CC(C)Nc1ccnc(N2CCN(Cc3cccs3)C(CCO)C2)n1,2.641,361.505,4,2,7,2.635,92.76
1,TEST_001,COc1cc(=O)n(-c2ccccc2)cc1C(=O)N1CCC2(CC1)OCCO2,0.585,370.399,5,0,3,0.585,68.31
2,TEST_002,Cc1cccc(NC(=N)/N=c2\nc(O)c(Cc3ccccc3)c(C)[nH]2)c1,4.276,347.414,4,4,5,4.290,92.86
3,TEST_003,O=C(c1nc2ncccn2n1)N1CCCn2cc(-c3ccccc3)nc21,1.795,345.358,5,0,2,1.795,81.21
4,TEST_004,CCN1CCN(C(=O)c2cc3c(=O)n4cc(C)ccc4nc3n2C)CC1,1.219,353.418,4,0,2,0.169,61.15


In [ ]:
train.shape, test.shape  #총 3498개의 데이터 (엄청 적음..) -> 총 483개의 데이터를 맞추기  (10%를)

((3498, 11), (483, 9))

In [ ]:
3498/483

7.24223602484472

In [ ]:
train.tail(10)

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
3488,TRAIN_3488,COc1ccc(C)cc1/N=c1/[nH]cnc2c1oc1ccccc12,2.874,3.733,3.652,305.331,3,1,2,3.652,59.12
3489,TRAIN_3489,[H][C@@]12C[C@@H](c3nccn3-c3cccc(C)c3)N3CCC[C@...,27.584,29.332,3.691,428.526,4,0,4,3.251,50.60
3490,TRAIN_3490,Cn1c(=O)c(-c2ccc3ccccc3n2)cc2cc(Br)ccc21,0.338,6.578,4.300,365.223,2,0,1,4.564,33.20
3491,TRAIN_3491,Cc1ccc(-n2ccnc2S(=O)(=O)Cc2noc(C)n2)c(C)c1,0.070,85.310,2.042,332.378,5,0,4,2.042,99.26
3492,TRAIN_3492,CN1CC2(CCN(C(=O)NC3CCCCC3)CC2)CC1C(=O)N1CCOCC1,82.271,83.464,1.073,392.536,4,1,2,-0.484,65.12
3493,TRAIN_3493,Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl,1.556,3.079,3.409,396.195,3,1,5,3.409,64.74
3494,TRAIN_3494,CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...,35.560,47.630,1.912,359.381,4,1,3,1.844,77.37
3495,TRAIN_3495,CCOC(=O)CCCc1nc2cc(N)ccc2n1C,56.150,1.790,1.941,261.320,3,1,6,2.124,70.14
3496,TRAIN_3496,Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl,0.030,2.770,0.989,284.696,5,1,5,0.989,91.51
3497,TRAIN_3497,COc1ccc(-c2nc(Cc3ccccc3)sc2C)cc1,0.450,2.650,4.321,295.399,2,0,4,4.321,50.36


In [ ]:
test.tail(10)

,id,SMILES,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
473,TEST_473,O=C1[C@@H]2CCCN2/C(=N/c2ccccc2)N1c1ccccc1,3.261,291.347,2,0,2,3.261,35.90
474,TEST_474,N#Cc1ccc(Oc2ccc(NC(=O)NCc3cc[nH]n3)cn2)cc1,2.016,334.332,5,3,5,2.016,115.72
475,TEST_475,O=C(Nc1ccc(-c2nc3ccccc3[nH]2)cc1)c1ccnc(-n2cnn...,2.590,381.390,5,2,4,2.533,101.38
476,TEST_476,O=C(c1ccccc1C1CCNC1)N1CCC2(C=Cc3ccccc32)CC1,3.088,358.476,2,1,2,2.803,32.34
477,TEST_477,COc1ccc(N(C)C(=O)c2cc(S(=O)(=O)N3CCc4ccccc43)c...,3.147,455.527,5,0,6,3.147,89.46
478,TEST_478,CCc1noc(CC)c1CC(=O)NCC1(CC)CCCCC1,4.207,306.443,2,1,7,4.207,55.13
479,TEST_479,CC(=O)N1CCC2(CC1)OC(=O)C(C)=C2C(=O)N1CCN(C)CC1,-0.608,335.398,5,0,1,-1.736,70.16
480,TEST_480,CC(C)NC(=O)CN1C(=O)c2ccccc2N2C(=O)c3ccccc3C12,1.792,349.383,3,1,3,1.792,69.72
481,TEST_481,Cn1cc(Br)c(=O)c(NC(=O)c2ccc(O)cc2F)c1,0.790,341.132,3,2,2,0.423,69.64
482,TEST_482,CC(C)C(CCN1CCN(C)CC1)c1ccco1,2.782,250.380,2,0,5,0.606,19.62


# 데이터 EDA

In [ ]:
train.info(), test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3498 entries, 0 to 3497
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          3498 non-null   object 
 1   SMILES                      3498 non-null   object 
 2   MLM                         3498 non-null   float64
 3   HLM                         3498 non-null   float64
 4   AlogP                       3496 non-null   float64
 5   Molecular_Weight            3498 non-null   float64
 6   Num_H_Acceptors             3498 non-null   int64  
 7   Num_H_Donors                3498 non-null   int64  
 8   Num_RotatableBonds          3498 non-null   int64  
 9   LogD                        3498 non-null   float64
 10  Molecular_PolarSurfaceArea  3498 non-null   float64
dtypes: float64(6), int64(3), object(2)
memory usage: 300.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 483 entries, 0 to 482
Data columns (to

(None, None)

In [ ]:
train['SMILES'].nunique() # 총 3497 데이터들 중에서 28개 데이터 겹침

3471

In [ ]:
len(train['SMILES'][0])

47

In [ ]:
train['SMILES'].unique().tolist() # 유사도 기준으로 화합물 분자식을 분리하기?

['CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC',
 'Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1',
 'CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1',
 'Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC5)CC4)c3C)nn2)cc1',
 'Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2',
 'COc1c(F)c(F)cc2c(=O)c(C(=O)NCCCN3CCCC3=O)cn(C3CC3)c12',
 'CN(C)c1cccc2c(S(=O)(=O)NC(CC(=O)O)C(=O)O)cccc12',
 'C1=Cn2nc(/C=C/c3cccs3)nc2-c2ccccc2O1',
 'COc1ccc2nc(C3=CCCN(C)C3)n(C)c2c1',
 'CCN1CCCC1CNC(=O)C1CCCN(c2ncnc3[nH]cnc23)C1',
 'CN(C1CCNCC1)S(=O)(=O)c1ccc(Cl)s1',
 'CC1CN(Cc2cncs2)CCN1CC(C)(C)O',
 'Nc1nc(N2CCN(c3ccccc3Cl)CC2)nc2ccccc12',
 'CC(C)c1ccc(CCCNS(C)(=O)=O)cc1',
 'Oc1cc(-c2ccc(-c3ccc(Br)cc3)cc2)nn1-c1ccccn1',
 'COc1cc(Nc2ncc(-c3cnn(C)c3)c(Nc3ccc4c(c3)CNCC4)n2)cc(OC)c1',
 'Cc1[nH]c(=O)[nH]c(=O)c1S(=O)(=O)NCC1CCCO1',
 'COCCN(CC1CCCN(C2Cc3ccccc3C2)C1)C(=O)Cc1c[nH]c2ccccc12',
 'O=C(O)C1C(c2ccccc2)C(C(=O)O)C1c1ccccc1',
 'CCC(CO)Nc1ncc2cc3ccccc3nc2c1C#N',
 'CNC(=O)c1cc(Cl)cc(C)c1NC(=O)c1cc(Br)nn1-c1c(F)c(F)cc(F)c1F',
 'CCCc1nn2c(N)c(C#N)nnc2c



---


# Feature Engineering

- LogP : 단순 화학 물질의 지용성 지표
- alogP : 화학 물질의 물리적 특성도 고려한 지용성 지표
- LogD : 물질의 이온화 상태까지 고려한 분배

In [ ]:
train_ft = train.copy()
test_ft = test.copy()

In [ ]:
train_ft.columns

Index(['id', 'SMILES', 'MLM', 'HLM', 'AlogP', 'Molecular_Weight',
       'Num_H_Acceptors', 'Num_H_Donors', 'Num_RotatableBonds', 'LogD',
       'Molecular_PolarSurfaceArea', 'SMILES_len'],
      dtype='object')

### SMILES관련
- 이 화합물의 분자가 어떤 특성을 가지고 있는지 smiles를 통해서 확인

- 구조 : 원자 및 결합 정보 (원자 종류, 수소 수, 이중결합, 삼중 결합) / 특정 기능성 기기 및 작용기의 존재 여부 파악
- 물리 : 화합물의 분자량, 로간 P값(지용성 표시), 극성지표 / 분자 디스크립터 생성 2D, 3D의 분자 디스크립터 생성하여 관계 파악
- 화학 : 함수군 및 치환기 정보 기반, 화합물의 화학적 반응의 특성 / 반응성 예측 (특정 화학적 변화, 반응)
- 생물 : 생체 이용도 예측, 화합물의 생물활성 정보 고려, 생물학적 활동 간의 연계


- 원자 정보 : 어떤 원소들이 unique 값으로 있는지 파악하기

In [ ]:
for smiles in train_ft['smiles'] :
    smiles

In [ ]:
# 분자식의 길이

train_ft['SMILES']


0         CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC
1                    Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1
2                        CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1
3       Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...
4                     Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2
                              ...                        
3493       Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl
3494    CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...
3495                         CCOC(=O)CCCc1nc2cc(N)ccc2n1C
3496                       Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl
3497                     COc1ccc(-c2nc(Cc3ccccc3)sc2C)cc1
Name: SMILES, Length: 3498, dtype: object

In [ ]:
for molecule in train_ft['SMILES'] :
    atom_list = []
    for atom in molecule :
        atom_list.append(atom)

    print(atom_list)


['C', 'C', 'O', 'c', '1', 'c', 'c', 'c', '(', 'C', 'N', 'C', '(', '=', 'O', ')', 'c', '2', 'c', 'c', '(', '-', 'c', '3', 's', 'c', '(', 'C', ')', 'n', 'c', '3', 'C', ')', 'n', '[', 'n', 'H', ']', '2', ')', 'c', 'c', '1', 'O', 'C', 'C']
['C', 'c', '1', 'n', 'c', '(', 'C', ')', 'c', '(', 'C', 'N', '2', 'C', 'C', '(', 'C', ')', 'C', '(', '=', 'O', ')', 'N', 'c', '3', 'c', 'c', 'c', 'c', 'c', '3', '2', ')', 's', '1']
['C', 'C', 'C', 'N', '1', 'C', 'C', 'N', '(', 'c', '2', 'n', 'n', '3', 'n', 'n', 'n', 'c', '3', 'c', '3', 'c', 'c', 'c', 'c', 'c', '2', '3', ')', 'C', 'C', '1']
['C', 'c', '1', 'c', 'c', 'c', '(', '-', 'c', '2', 'c', 'c', 'c', '(', '-', 'n', '3', 'n', 'c', '(', 'C', ')', 'c', '(', 'S', '(', '=', 'O', ')', '(', '=', 'O', ')', 'N', '4', 'C', 'C', 'N', '(', 'C', '5', 'C', 'C', 'C', 'C', 'C', '5', ')', 'C', 'C', '4', ')', 'c', '3', 'C', ')', 'n', 'n', '2', ')', 'c', 'c', '1']
['C', 'c', '1', 'c', 'c', 'c', '2', 'c', '(', 'c', '1', ')', 'N', '(', 'C', '(', '=', 'O', ')', 'c', '1', 

In [ ]:
all_atom_lists = []  # 모든 분자의 원자 리스트를 저장할 리스트
for molecule in train_ft['SMILES']:
    atom_list = []  # 각 분자의 원자들을 저장할 리스트
    for atom in molecule:
        atom_list.append(atom)  # 각 원자를 리스트에 추가
    all_atom_lists.append(atom_list)  # 분자별 원자 리스트를 모든 분자의 리스트에 추가

print("모든 분자의 원자 리스트:", all_atom_lists)



모든 분자의 원자 리스트: [['C', 'C', 'O', 'c', '1', 'c', 'c', 'c', '(', 'C', 'N', 'C', '(', '=', 'O', ')', 'c', '2', 'c', 'c', '(', '-', 'c', '3', 's', 'c', '(', 'C', ')', 'n', 'c', '3', 'C', ')', 'n', '[', 'n', 'H', ']', '2', ')', 'c', 'c', '1', 'O', 'C', 'C'], ['C', 'c', '1', 'n', 'c', '(', 'C', ')', 'c', '(', 'C', 'N', '2', 'C', 'C', '(', 'C', ')', 'C', '(', '=', 'O', ')', 'N', 'c', '3', 'c', 'c', 'c', 'c', 'c', '3', '2', ')', 's', '1'], ['C', 'C', 'C', 'N', '1', 'C', 'C', 'N', '(', 'c', '2', 'n', 'n', '3', 'n', 'n', 'n', 'c', '3', 'c', '3', 'c', 'c', 'c', 'c', 'c', '2', '3', ')', 'C', 'C', '1'], ['C', 'c', '1', 'c', 'c', 'c', '(', '-', 'c', '2', 'c', 'c', 'c', '(', '-', 'n', '3', 'n', 'c', '(', 'C', ')', 'c', '(', 'S', '(', '=', 'O', ')', '(', '=', 'O', ')', 'N', '4', 'C', 'C', 'N', '(', 'C', '5', 'C', 'C', 'C', 'C', 'C', '5', ')', 'C', 'C', '4', ')', 'c', '3', 'C', ')', 'n', 'n', '2', ')', 'c', 'c', '1'], ['C', 'c', '1', 'c', 'c', 'c', '2', 'c', '(', 'c', '1', ')', 'N', '(', 'C', '(', '=', 

In [ ]:
atom_char_lists = []

for i in all_atom_lists :
    for char in i :
        atom_char_lists.append(char)


In [ ]:
len(set(atom_char_lists)) #총 32개 -> 32개에 대해서 원핫 인코딩 해도 될수도?

32

In [ ]:
all_atom_lists #리스트들의 리스트

[['C',
  'C',
  'O',
  'c',
  '1',
  'c',
  'c',
  'c',
  '(',
  'C',
  'N',
  'C',
  '(',
  '=',
  'O',
  ')',
  'c',
  '2',
  'c',
  'c',
  '(',
  '-',
  'c',
  '3',
  's',
  'c',
  '(',
  'C',
  ')',
  'n',
  'c',
  '3',
  'C',
  ')',
  'n',
  '[',
  'n',
  'H',
  ']',
  '2',
  ')',
  'c',
  'c',
  '1',
  'O',
  'C',
  'C'],
 ['C',
  'c',
  '1',
  'n',
  'c',
  '(',
  'C',
  ')',
  'c',
  '(',
  'C',
  'N',
  '2',
  'C',
  'C',
  '(',
  'C',
  ')',
  'C',
  '(',
  '=',
  'O',
  ')',
  'N',
  'c',
  '3',
  'c',
  'c',
  'c',
  'c',
  'c',
  '3',
  '2',
  ')',
  's',
  '1'],
 ['C',
  'C',
  'C',
  'N',
  '1',
  'C',
  'C',
  'N',
  '(',
  'c',
  '2',
  'n',
  'n',
  '3',
  'n',
  'n',
  'n',
  'c',
  '3',
  'c',
  '3',
  'c',
  'c',
  'c',
  'c',
  'c',
  '2',
  '3',
  ')',
  'C',
  'C',
  '1'],
 ['C',
  'c',
  '1',
  'c',
  'c',
  'c',
  '(',
  '-',
  'c',
  '2',
  'c',
  'c',
  'c',
  '(',
  '-',
  'n',
  '3',
  'n',
  'c',
  '(',
  'C',
  ')',
  'c',
  '(',
  'S',
  '(',
  '=',
  '

In [ ]:
#화합물의 분자

#### 분자 스크립터 생성 (2d, 3d)



#### 1) 기본 2D 디스크립터 : rdkit chem

In [ ]:
!pip install rdkit-pypi


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 32.4 MB/s eta 0:00:00


In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem

def calculate_2d_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)

    if mol is None:
        raise ValueError("Invalid SMILES string")

    descriptors = {}

    # Basic 2D descriptors
    descriptors['MolecularWeight'] = Descriptors.MolWt(mol)
    descriptors['LogP'] = Descriptors.MolLogP(mol)

    # More descriptors can be added based on your needs

    return descriptors



In [ ]:
# Example SMILES string
example_smiles = train_ft['SMILES'][0]  # Replace this with your SMILES string

# Calculate 2D descriptors
descriptors = calculate_2d_descriptors(example_smiles)

# Print the calculated descriptors
for descriptor_name, value in descriptors.items():
    print(f"{descriptor_name}: {value}")

MolecularWeight: 400.50400000000013
LogP: 3.8774400000000018


In [ ]:
train_ft.head(10)

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,SMILES_len
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,117.37,3498
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,73.47,3498
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,62.45,3498
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,92.60,3498
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,42.43,3498
5,TRAIN_0005,COc1c(F)c(F)cc2c(=O)c(C(=O)NCCCN3CCCC3=O)cn(C3...,27.640,66.630,1.335,419.422,4,1,7,1.335,78.95,3498
6,TRAIN_0006,CN(C)c1cccc2c(S(=O)(=O)NC(CC(=O)O)C(=O)O)cccc12,82.107,69.250,1.954,547.707,7,4,9,0.464,144.42,3498
7,TRAIN_0007,C1=Cn2nc(/C=C/c3cccs3)nc2-c2ccccc2O1,2.673,25.121,2.967,293.343,3,0,2,2.967,68.18,3498
8,TRAIN_0008,COc1ccc2nc(C3=CCCN(C)C3)n(C)c2c1,74.519,71.471,-0.946,347.366,7,2,3,-0.723,104.89,3498
9,TRAIN_0009,CCN1CCCC1CNC(=O)C1CCCN(c2ncnc3[nH]cnc23)C1,73.417,76.291,1.259,357.453,5,2,5,-0.373,90.04,3498


#### 2) 2D/3D 통합 디스크립터
- 3D 디스크립터 : 분자의 3차원 공간적 정보를 나타내는 값

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem

def calculate_3d_descriptor(smiles):
    mol = Chem.MolFromSmiles(smiles)

    if mol is None:
        raise ValueError("Invalid SMILES string")

    # Generate 3D coordinates
    mol = Chem.AddHs(mol)  # Add hydrogens for accurate 3D calculation
    AllChem.EmbedMolecule(mol, randomSeed=42)  # Embed the molecule in 3D space

    # Calculate 3D descriptor: Predicted molecular volume
    descriptor_3d = AllChem.ComputeMolVolume(mol)

    return descriptor_3d

# Example SMILES string
example_smiles = train_ft['SMILES'][0]  # Replace this with your SMILES string

# Calculate 3D descriptor
descriptor_3d = calculate_3d_descriptor(example_smiles)

print(descriptor_3d)



361.6400000000001


- 2D, 3D 통합 디스크립터 : 분자의 구조 (2d) + 공간 (3d) 적 특성을 모두 고려함

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors

def calculate_2d_3d_combined_descriptor(smiles):
    mol = Chem.MolFromSmiles(smiles)

    if mol is None:
        raise ValueError("Invalid SMILES string")

    # Calculate 2D descriptors
    mw = Descriptors.MolWt(mol)  # Molecular weight
    logp = Descriptors.MolLogP(mol)  # LogP

    # Generate 3D coordinates
    mol = Chem.AddHs(mol)  # Add hydrogens for accurate 3D calculation
    AllChem.EmbedMolecule(mol, randomSeed=42)  # Embed the molecule in 3D space

    # Calculate 3D descriptor: Predicted molecular volume
    volume_3d = AllChem.ComputeMolVolume(mol)

    # Combine 2D and 3D descriptors into a dictionary
    combined_descriptor = {
        'MolecularWeight': mw,
        'LogP': logp,
        'Volume3D': volume_3d
    }

    return combined_descriptor



In [ ]:
# Example SMILES string
example_smiles = train_ft['SMILES'][0] # Replace this with your SMILES string

# Calculate combined 2D/3D descriptor
combined_descriptor = calculate_2d_3d_combined_descriptor(example_smiles)

print(combined_descriptor)

{'MolecularWeight': 400.50400000000013, 'LogP': 3.8774400000000018, 'Volume3D': 361.6400000000001}


#### 3) Morgan Fingerprints

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem

def calculate_morgan_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)

    if mol is None:
        raise ValueError("Invalid SMILES string")

    radius = 2  # Morgan fingerprint radius
    n_bits = 2048  # Number of bits in the fingerprint

    fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)

    return fingerprint


In [ ]:

# Example SMILES string
example_smiles = train_ft['SMILES'][0]   # Replace this with your SMILES string

# Calculate Morgan fingerprint
morgan_fingerprint = calculate_morgan_fingerprint(example_smiles)

for i in morgan_fingerprint:
    print(i)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0


#### 4) MACCS Keys

In [ ]:
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors

def calculate_maccs_keys(smiles):
    mol = Chem.MolFromSmiles(smiles)

    if mol is None:
        raise ValueError("Invalid SMILES string")

    maccs_keys = rdMolDescriptors.GetMACCSKeys(mol)

    return maccs_keys

# Example SMILES string
example_smiles = train_ft['SMILES'][6]   # Replace this with your SMILES string

# Calculate MACCS keys
maccs_keys = calculate_maccs_keys(example_smiles)

print(maccs_keys)


AttributeError: ignored

#### 5) RDKit Fingerprints

In [ ]:
from rdkit import Chem
from rdkit.Chem import RDKFingerprint

def calculate_rdk_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)

    if mol is None:
        raise ValueError("Invalid SMILES string")

    fingerprint = RDKFingerprint(mol)

    return fingerprint

# Example SMILES string
example_smiles = 'CCO'  # Replace this with your SMILES string

# Calculate RDKit fingerprint
rdk_fingerprint = calculate_rdk_fingerprint(example_smiles)

rdk_fingerprint

#### 6) Physicochemical Descriptors

- 2d, 3d 디스크립터로부터 최대한 많은 요인들을 추축

### Lipinski의 5가지 법칙 (5가지 요소)
- 신약 후보 분자의 경구 생체이용도를 예측하기 위한 규칙 :구강복용하는 약들은 아래의 4가지 중 하나라도 벗어나서는 안됨
- Lipinski 5
    - 분자량 : 500dalton 이하
    - 로그P : 5 이하 (partition coefficient)
    - 수소키수용체 HbA : 10개 이하
    - 수소결합드너 HbD : 5개 이하
    - 회전가능결합수 RB : 10개 이하

- cLogP (코왜이노용해도)

In [ ]:
train_ft.columns

Index(['id', 'SMILES', 'MLM', 'HLM', 'AlogP', 'Molecular_Weight',
       'Num_H_Acceptors', 'Num_H_Donors', 'Num_RotatableBonds', 'LogD',
       'Molecular_PolarSurfaceArea', 'SMILES_len'],
      dtype='object')

In [ ]:
train_ft['Num_H_Donors'].max()

10

In [ ]:
train_ft['Num_H_Acceptors'].max()

15

In [ ]:
train_ft['Num_RotatableBonds'].max()

38

In [ ]:
train_ft['Molecular_Weight'].max()

1360.467

### 수용성, 투과성, 극성


In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

def calculate_metabolism_stability(smiles):
    mol = Chem.MolFromSmiles(smiles)

    if mol is None:
        raise ValueError("유효하지 않은 SMILES 문자열")

    # Lipinski의 5법칙 디스크립터 계산
    mw = Descriptors.MolWt(mol)  # 분자량
    logp = Descriptors.MolLogP(mol)  # 로그P
    hba = Lipinski.NumHAcceptors(mol)  # 수소 키 수용체
    hbd = Lipinski.NumHDonors(mol)  # 수소 결합 드너
    rb = Lipinski.NumRotatableBonds(mol)  # 회전 가능한 결합 수

    # cLogP 계산 (LogP의 복잡성 지표)
    clogp = Descriptors.MolMR(mol) / 160.38

    # 대사안정성 평가 (예시)
    metabolism_stability_score = mw + logp - hba - hbd - rb + clogp

    return metabolism_stability_score

# 예제 SMILES 문자열
example_smiles = 'CCO'  # 여기에 자신의 SMILES 문자열을 입력하세요

# 대사안정성 평가
metabolism_stability_score = calculate_metabolism_stability(example_smiles)

print("대사안정성 평가 점수:", metabolism_stability_score)


대사안정성 평가 점수: 44.14715979548573


In [ ]:
#    combined_descriptor = {
#         '분자량': mw,
#         '로그P': logp,
#         '위상학적 극성 표면 면적': tpsa,
#         '수소 키 수용체': hba,
#         '수소 결합 드너': hbd,
#         '회전 가능한 결합 수': rb,


#         '3D 부피': volume_3d,
#         # '용매 접근 가능 표면 면적': sasa,

#         '공식 전하량': charge,
#         '질소 원자 수': num_nitrogens,
#         '산소 원자 수': num_oxygens,
#         '방향족 환 수': num_aromatic_rings

        # 'CYP450 인도효소 접근 가능성': cyp450_accessibility

In [ ]:
from rdkit.Chem import AllChem, Descriptors, Lipinski

def calculate_metabolism_stability_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)

    if mol is None:
        raise ValueError("유효하지 않은 SMILES 문자열")

    # 2차원 디스크립터 계산
    mw = Descriptors.MolWt(mol)  # 분자량
    logp = Descriptors.MolLogP(mol)  # 로그P
    tpsa = Descriptors.TPSA(mol)  # 위상학적 극성 표면 면적

    # Lipinski의 5법칙 디스크립터 계산
    hba = Lipinski.NumHAcceptors(mol)  # 수소 키 수용체
    hbd = Lipinski.NumHDonors(mol)  # 수소 결합 드너
    rb = Lipinski.NumRotatableBonds(mol)  # 회전 가능한 결합 수

    # 3D 좌표 생성
    mol = Chem.AddHs(mol)  # 정확한 3D 계산을 위해 수소 추가
    AllChem.EmbedMolecule(mol, randomSeed=42)  # 3D 공간에 분자 임베딩

    # Calculate CYP450 인도효소 접근 가능성
    cyp450_accessibility = AllChem.CalcCYP450PAC(mol)

    # 다양한 2D와 3D 디스크립터를 사전 형태로 결합
    combined_descriptor = {
        '분자량': mw,
        '로그P': logp,
        '위상학적 극성 표면 면적': tpsa,
        '수소 키 수용체': hba,
        '수소 결합 드너': hbd,
        '회전 가능한 결합 수': rb,
        'CYP450 인도효소 접근 가능성': cyp450_accessibility
        # 필요한 경우 더 많은 디스크립터 추가
    }

    return combined_descriptor

# 예제 SMILES 문자열
example_smiles = 'CCO'  # 여기에 자신의 SMILES 문자열을 입력하세요

# 대사안정성 디스크립터 계산
metabolism_stability_descriptors = calculate_metabolism_stability_descriptors(example_smiles)

print(metabolism_stability_descriptors)


AttributeError: ignored

In [ ]:
from rdkit import Chem
from rdkit.Chem import Lipinski, rdMolDescriptors

def calculate_additional_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)

    if mol is None:
        raise ValueError("유효하지 않은 SMILES 문자열")

    # 질소 원자 수 계산
    num_nitrogens = rdMolDescriptors.CalcNumN(mol)

    # 산소 원자 수 계산
    num_oxygens = rdMolDescriptors.CalcNumO(mol)

    # 방향족 환 수 계산
    num_aromatic_rings = Lipinski.NumAromaticRings(mol)

    additional_descriptors = {
        '질소 원자 수': num_nitrogens,
        '산소 원자 수': num_oxygens,
        '방향족 환 수': num_aromatic_rings
    }

    return additional_descriptors

# 예제 SMILES 문자열
example_smiles = 'CCO'  # 여기에 자신의 SMILES 문자열을 입력하세요

# 추가 디스크립터 계산
additional_descriptors = calculate_additional_descriptors(example_smiles)

print(additional_descriptors)


AttributeError: ignored

In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

def calculate_additional_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)

    if mol is None:
        raise ValueError("유효하지 않은 SMILES 문자열")

    # 질소 원자 수 계산
    num_nitrogens = Descriptors.NumNitrogens(mol)

    # 산소 원자 수 계산
    num_oxygens = Descriptors.NumOxygens(mol)

    # 방향족 환 수 계산
    num_aromatic_rings = Lipinski.NumAromaticRings(mol)

    additional_descriptors = {
        '질소 원자 수': num_nitrogens,
        '산소 원자 수': num_oxygens,
        '방향족 환 수': num_aromatic_rings
    }

    return additional_descriptors

# 예제 SMILES 문자열
example_smiles = 'CCO'  # 여기에 자신의 SMILES 문자열을 입력하세요

# 추가 디스크립터 계산
additional_descriptors = calculate_additional_descriptors(example_smiles)

print(additional_descriptors)


AttributeError: ignored

In [ ]:

# 예제 SMILES 문자열
example_smiles = train_ft['SMILES'][0]  # 여기에 자신의 SMILES 문자열을 입력하세요

# 대사안정성 디스크립터 계산
metabolism_stability_descriptors = calculate_metabolism_stability_descriptors(example_smiles)

print(metabolism_stability_descriptors)

AttributeError: ignored

### MLM, HLM 관련 (target)
- MLM : 평균 37, 0~131까지
- HLM : 평균 53, 0~135까지

- 총 target값이 두개

In [ ]:
# 대사안정성의 quanitle을 범주화하기
# 0~131까지 /

train_ft['MLM'].min(), train_ft['MLM'].max(), train_ft['MLM'].sum() / len(train_ft['MLM'])

(0.0, 131.72, 37.38474185248713)

In [ ]:
train_ft['HLM'].min(), train_ft['HLM'].max(), train_ft['HLM'].sum() / len(train_ft['HLM'])

(0.0, 135.336, 53.0902061177816)

### AlogP, LogD 관련

# 학습 및 예측 (baseline) : DL